# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [95]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)
# To establish connection and begin executing queries, need a session


#### Create Keyspace

In [96]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }    
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [97]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [98]:
create_table_base_query = """
    CREATE TABLE IF NOT EXISTS {table_name} (
        artist text,
        firstName text,
        gender text,
        itemInSession int,
        lastName text,
        length float,
        level text,
        location text,
        sessionId int,
        song text,
        userId int,
        
        {primary_key}
    )
"""

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


In [99]:
create_table_events_session = create_table_base_query.format(
    table_name = "events_session",
    primary_key = "PRIMARY KEY (sessionId, itemInSession)"
)

try:
    session.execute(create_table_events_session)
except Exception as e:
    print(e)

In [100]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events_session (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(
            query, 
            (
                line[0] 
                , line[1] 
                , line[2] 
                , int(line[3]) 
                , line[4] 
                , float(line[5]) 
                , line[6] 
                , line[7] 
                , int(line[8]) 
                , line[9] 
                , int(line[10])
            )
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [101]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_1 = """
    select 
        artist, 
        song, 
        length
    from events_session
    where sessionId = 338 
        and itemInSession = 4
"""

try:
    rows = session.execute(query_1)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [102]:
create_table_events_user_session = create_table_base_query.format(
    table_name = "events_user_session",
    primary_key = "PRIMARY KEY (userId, sessionId, itemInSession)"
)

try:
    session.execute(create_table_events_user_session)
except Exception as e:
    print(e)


In [103]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events_user_session (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(
            query, 
            (
                line[0] 
                , line[1] 
                , line[2] 
                , int(line[3]) 
                , line[4] 
                , float(line[5]) 
                , line[6] 
                , line[7] 
                , int(line[8]) 
                , line[9] 
                , int(line[10])
            )
        )

In [104]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_2 = """
    select 
        artist, 
        song,
        firstName,
        lastName
    from events_user_session
    where userId = 10 
        and sessionId = 182
    order by sessionId, itemInSession
"""

                    

try:
    rows = session.execute(query_2)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [105]:
create_table_events_song = create_table_base_query.format(
    table_name = "events_song",
    primary_key = "PRIMARY KEY (song, sessionId, itemInSession)"
)

try:
    session.execute(create_table_events_song)
except Exception as e:
    print(e)


In [106]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO events_song (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(
            query, 
            (
                line[0] 
                , line[1] 
                , line[2] 
                , int(line[3]) 
                , line[4] 
                , float(line[5]) 
                , line[6] 
                , line[7] 
                , int(line[8]) 
                , line[9] 
                , int(line[10])
            )
        )

In [107]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_3 = """
    select 
        firstName, 
        lastName
    from events_song
    where song = 'All Hands Against His Own'
"""

                    

try:
    rows = session.execute(query_3)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Sara', lastname='Johnson')
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [108]:
## TO-DO: Drop the table before closing out the sessions

session.execute("DROP TABLE events_session")
session.execute("DROP TABLE events_user_session")
session.execute("DROP TABLE events_song")


### Close the session and cluster connection¶

In [109]:
session.shutdown()
cluster.shutdown()